# config parameters

In [ ]:
route = 1

In [ ]:
date  = '2016-06-13 08:10:00'

# script to test bunching

In [ ]:
import pandas as pd

In [ ]:
import os

In [248]:
import numpy as np

In [ ]:
files = os.listdir('Pairs')

# Create Training data from different pair files

In [ ]:
df = pd.DataFrame()

In [ ]:
for file in files:
    data = pd.read_csv('Pairs/'+file)
    #print(data.shape)
    df = df.append(data)

In [ ]:
df = df[df['travel_time']<=100]

In [ ]:
df = df[['route_id','s','d','start_time','travel_time','weekday']]

In [ ]:
df.head()

In [ ]:
df.to_csv('Pairs/training_data')

In [ ]:
import datetime

In [ ]:
date_time1 = datetime.datetime.strptime(date,'%Y-%m-%d %H:%M:%S')

In [ ]:
time_delta = datetime.timedelta(minutes=10)

In [ ]:
date_time2 = date_time + time_delta

In [ ]:
print (date_time1,date_time2)

In [ ]:
test_file_name = date_time1.date()

In [ ]:
df = pd.read_csv(str(test_file_name))

In [ ]:
df.head()

In [ ]:
df['date_time'] = pd.to_datetime(df['date_time'])

In [ ]:
mask = (df['date_time']>date_time1) & (df['date_time']<date_time2)

In [ ]:
df = df[mask]

In [ ]:
#df13

In [ ]:
First = df.drop_duplicates(['device_id'],keep = 'first')
Last = df.drop_duplicates(['device_id'],keep = 'last')

In [ ]:
devices = df['device_id'].unique()

In [ ]:
mapping = {}
i=1

In [ ]:
for d in devices:
    mapping[d]=i
    i = i+1

In [ ]:
mapping


In [ ]:
First = First.replace({'device_id':mapping})
Last = Last.replace({'device_id':mapping})

In [ ]:
First = First.sort_values('device_id')

In [ ]:
Last = Last.sort_values('device_id')

In [ ]:
First['real_time']=First['date_time']
Last['real_time']=Last['date_time']

In [ ]:
First['date_time'] = First['date_time'].astype('str').map(lambda x : x.split()[1])
Last['date_time'] = Last['date_time'].astype('str').map(lambda x : x.split()[1])

In [ ]:
First['date_time'] = First['date_time'].map(lambda x : get_hour_min(x))
Last['date_time'] = Last['date_time'].map(lambda x : get_hour_min(x))

In [ ]:
First.head()

In [ ]:
First['date_time'] = First['date_time']/60
First['date_time'] = First['date_time'].map(lambda x : round(x,2))
Last['date_time'] = Last['date_time']/60
Last['date_time'] = Last['date_time'].map(lambda x : round(x,2))

In [ ]:
from geopy.distance import vincenty
import json

In [ ]:
dist = 

In [ ]:
with open('../K-4/june2_thursday/b_stops.txt') as file:
    b_stops = json.load(file)

In [ ]:
stops_dic = {}

In [ ]:
for i in range(len(devices)):
    first_distance = 100000
    last_distance = 100000
    
    first_stop =1
    last_stop =1
    
    first_s_lat = First[First['device_id']==i+1]['lat']
    first_s_long = First[First['device_id']==i+1]['long']
    last_s_lat = Last[Last['device_id']==i+1]['lat']
    last_s_long = Last[Last['device_id']==i+1]['long']
    #print(last_s_lat)
    
    for j in range(len(b_stops)):
        d_lat = b_stops[j]['latlons'][0]
        d_long = b_stops[j]['latlons'][1]
        
        #print(d_long)
        #new_first_distance = vincenty((19.0760,72.8777),(18.5204,73.8567)).meters
        new_first_distance = vincenty((first_s_lat,first_s_long),(d_lat,d_long)).meters
        #new_last_distance = vincenty((last_s_lat,last_s_long),(d_lat,d_long)).meters
         
        new_first_distance =100
        new_last_distance = 100
        if(new_first_distance<first_distance):
            first_distance = new_first_distance
            first_stop = j+1
        if(new_last_distance<last_distance):
            last_distance = new_last_distance
            last_stop = j+1
    if(first_stop<last_stop):
        direction = 1
    else:
        direction = 0
    new_dic = {'direction':direction,'b_stop':first_stop}
    
    stops_dic[i+1]= new_dic

In [ ]:
stops_dic

In [ ]:
First['lat'][First['device_id']==i+1]

In [ ]:
stops_dic[2]['direction']=1

In [ ]:
stops_dic

In [ ]:
stops_dic.items()

In [ ]:
up = []
down = []

In [ ]:
for i,j in stops_dic.items():
    if j['direction']==1:
        up.append(i)
    else:
        down.append(i)
    

In [ ]:
up

In [ ]:
down

In [ ]:
stops_dic[1]['b_stop']=5

In [ ]:
stops_dic[3]['b_stop']=10

In [ ]:
stops_dic[4]['b_stop']=8

In [ ]:
down= sorted(down,key=lambda x : stops_dic[x]['b_stop'])

In [ ]:
down

In [ ]:
def get_hour_min(x):
    from datetime import datetime
    date_obj = datetime.strptime(x, '%H:%M:%S')
    
    #date = date_obj.strftime("%H:%M")
    return date_obj.hour * 60 +date_obj.minute

In [ ]:
X_test = list()

In [252]:
ith_bus_travel_time = []
jth_bus_travel_time = []
for i in range(len(down)-1):
    X_test_i = list()
    X_test_j = list()
    i_b_stop = stops_dic[down[i]]['b_stop']
    j_b_stop = stops_dic[down[i+1]]['b_stop']
    #predict travel time for all bus stops
    #compare travel time of ith and jth bus stop
    #find bunching
    k = j_b_stop+1
    z=k
    while(k<=len(b_stops)):
        start_time = First[First['device_id']==down[i]]['date_time']
        list_i = list([route,i_b_stop,k,start_time])
        list_j = list([route,j_b_stop,k,start_time])
    X_test_i.append(list_i)
    X_test_j.append(list_j)
        
    X_test_i = np.array(X_test_i)
    X_test_j = np.array(X_test_j)
        
    import pickle
    from sklearn.linear_model import LinearRegression
        
    model = pickle.load(open('model','rb'))
    y_i = model.predict(X_test_i)
    y_j = model.predict(X_test_j)
    
    t=0
    while(k<=len(b_stops)):
        diff = abs(y_i[t]-y_j[t])
        if diff >=10:
            print("Bunching at bus stop {}".format(z+t))
        
    
        
        
        
    
    
    
        

KeyboardInterrupt: 